In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score
%matplotlib inline
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
import nltk.corpus
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer,PorterStemmer
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('tagsets')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Tanriverdi\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Tanriverdi\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Tanriverdi\AppData\Roaming\nltk_data...
[nltk_data] Downloading package tagsets to
[nltk_data]     C:\Users\Tanriverdi\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping help\tagsets.zip.


True

In [2]:
tweets_d = pd.read_csv('Data/tweets_disc_cleaned.csv')
tweets_c = pd.read_csv('Data/tweets_ctrl_cleaned.csv')
data = pd.concat([tweets_d])
data

,date,id,content,likes,retweets,disc_cat,Unnamed: 0.1,disc_cat_num
0,28 February 2023,1630587451436683264,bitch lmfaooooo gone rock sock ass,0,0,gender,NaN,2
1,28 February 2023,1630587450710958081,nvr vw person simpli cost repair hassl get don...,0,0,gender,NaN,2
2,28 February 2023,1630587450673319940,hate wen bitch say stop act care df said act l...,0,0,gender,NaN,2
3,28 February 2023,1630587450547634179,dnc pay well ask brooklyn dad hes lucki get pe...,0,0,gender,NaN,2
4,28 February 2023,1630587449503264769,sinong bitch ba yung hindi marunong mag sorri ...,0,0,gender,NaN,2
...,...,...,...,...,...,...,...,...
210913,28 February 2023,1630401406883561472,NaN,3,0,mental_health,105706.0,4
210914,28 February 2023,1630401254798118913,make sens,1,0,mental_health,105707.0,4
210915,28 February 2023,1630401175995527169,mean make sens miz host,0,0,mental_health,105708.0,4
210916,28 February 2023,1630400994222948352,disrespect key lime pieoh use lubric,1,0,mental_health,105709.0,4


In [3]:
print(data.dtypes)

date             object
id                int64
content          object
likes             int64
retweets          int64
disc_cat         object
Unnamed: 0.1    float64
disc_cat_num      int64
dtype: object


In [4]:
data['content'] =  data['content'].astype(str)

# MODELLING

## NLP 

### Creating the Corpus

In [5]:
contents = []
corpus = []
data['content'].dropna()
for tweet in data['content']:
    contents.append(tweet)
    corpus.append(nltk.sent_tokenize(tweet))


In [6]:
corpus

[['bitch lmfaooooo gone rock sock ass'],
 ['nvr vw person simpli cost repair hassl get done someth break solidifi disdain compani'],
 ['hate wen bitch say stop act care df said act lil bih'],
 ['dnc pay well ask brooklyn dad hes lucki get per check sinc hes part soy boy divorc angri bitch'],
 ['sinong bitch ba yung hindi marunong mag sorri gago ka nic'],
 ['nan'],
 ['mean son bitch pleas call us latinx offens'],
 ['nan'],
 ['googl bitch clown motherfuck better take sensit ass back myspac'],
 ['boss shut bitch'],
 ['bitch sus sinc day meet'],
 ['pep prove littl bitch'],
 ['use sweet innoc slow ass bitch'],
 ['last long'],
 ['yeahyou need retwist bitch'],
 ['start back bitch gone cri'],
 ['thank good block evil bitch earli time account forc follow either also sinc happen twitter must said follow alway mean endors'],
 ['bitch serious use art pull bitch lmfao'],
 ['flip fist bitch free vega xxxl carniv xl'],
 ['goat cannot knocc da hustl prize pick done bitch'],
 ['mug back bitch thought g

# Discriminatory words

In [7]:
sex_discriminatory_words = [
    "bitch", "whore", "slut", "cunt", "ho", "wench", "skank", "tart", "floozy", "prostitute", 
    "call girl", "streetwalker", "escort", "hooker", "gold digger", "slag", "tramp", "harlot", "madam", 
    "lady of the night", "mistress", "sugar baby", "beard", "chick", "bird", "doll", "babe", "fox", 
    "cutie", "hottie", "piece of ass", "eye candy", "trophy wife", "sex kitten", "man-whore", "gigolo", 
    "male escort", "sugar daddy", "john", "pimp", "player", "stud", "womanizer", "slayer", "Don Juan", 
    "Casanova", "pick-up artist", "seduction community", "incel", "volcel", "MGTOW", "PUA", "red pill", 
    "blue pill", "friendzone", "nice guy", "alpha", "beta", "omega", "hypergamy", "feminazi", "misandrist", 
    "misogynist", "toxic masculinity", "fragile masculinity", "mansplain", "manterrupt", "hepeating", 
    "manspread", "bro", "dude", "guy", "man"
]
gender_discriminatory_words = [
    "fag", "dyke", "tranny", "shemale", "chick with a dick", "he-she", "it", "sissy", "butch", "queer", 
    "homo", "gay", "lesbo", "lesbian", "bisexual", "pansexual", "asexual", "aromantic", "non-binary", 
    "genderqueer", "genderfluid", "transgender", "transvestite", "crossdresser",
    "bitch", "slut", "whore", "prostitute", "harlot", "hooker", "skank", "tramp", "gold digger", 
    "ho bag", "cum dumpster", "cock tease", "man hater", "ball buster", "nag", "shrew", "virgin", 
    "prude", "dyke", "fag", "faggot", "sissy", "emasculated", "emasculate", "sissies", "effeminate", 
    "effeminate man", "masculine woman", "manly woman", "tomboy", "butch", "girly", "manly girl", 
    "girly man"
]
race_discriminatory_words = [
    "n-word", "nigga", "nigger", "spic", "chink", "wetback", "jap", "kike", "raghead", "gook", "cracker", "honky", "redneck", 
    "white trash", "hillbilly", "coon", "sambo", "boy", "mammy", "darkie", "half-breed", "mulatto", "octoroon", 
    "quadroon", "yellow", "curry muncher", "abo", "gyppo", "wog", "pikey", "gypsy", "tinker", "papist", 
    "heathen", "infidel", "crusader", "pagan"
]
health_discriminatory_words = [
    "crazy", "lunatic", "psycho", "nutjob", "schizo", "retarded", "handicapped", "invalid", "cripple", 
    "midget", "dwarf", "giant", "obese", "anorexic", "bulimic", "cancer stick", "crippled", "spastic", 
    "epileptic", "insane", "mad", "insane asylum", "loony bin", "nut house", "funny farm", "sanitarium", 
    "booby hatch", "rubber room", "straightjacket", "mental institution", "mental hospital", "snake pit"
]
mental_health_discriminatory_words = [
    "crazy", "lunatic", "psycho", "nutjob", "schizo", "retarded", "insane", "mad", "insane asylum", 
    "loony bin", "nut house", "funny farm", "sanitarium", "booby hatch", "rubber room", "straightjacket", 
    "mental institution", "mental hospital", "snake pit", "depressed", "anxious", "bipolar", "manic", 
    "schizophrenic", "borderline", "narcissistic", "antisocial", "OCD", "PTSD", "ADHD", "autistic", 
    "sociopath", "psychopath", "crazy person", "insane person", "lunatic person", "mental patient", 
    "madman", "madwoman", "crazy lady", "crazy guy", "crazy person", "psycho killer", "maniac", "lunatic fringe"
]
discriminatory_words = {'sex': sex_discriminatory_words, 'gender' : gender_discriminatory_words, 'race' : race_discriminatory_words, 'health' : health_discriminatory_words, 'mental_health' : mental_health_discriminatory_words}

### Tokenizing the Corpus

In [8]:
def get_top_discriminatory_words(data, discriminatory_words, disc_col='disc_cat', text_col='content'):
    top_words = {}
    for key in discriminatory_words:
        if key in data[disc_col].unique():
            if data[text_col].dtype == 'object':
                data[text_col] = data[text_col].astype(str)
            mask = data[disc_col].str.contains(key, case=False)
            if not mask.any():
                continue
            words = data.loc[mask, text_col].str.split(expand=True).stack().value_counts()
            top_words[key] = words[words.index.isin(discriminatory_words[key])][:50]
    return top_words

In [9]:
dword_freq = get_top_discriminatory_words(data, discriminatory_words)


In [10]:
len(dword_freq)

5

In [11]:
dword_freq

{'sex': man             2490
 guy             2380
 mansplain       1552
 bro             1131
 player          1116
 misogynist      1067
 manterrupt      1053
 incel           1041
 dude            1016
 escort           969
 manspread        918
 slayer           910
 slag             875
 feminazi         833
 ho               830
 cunt             726
 babe             707
 beta             695
 stud             559
 madam            545
 alpha            513
 bird             510
 beard            485
 fox              484
 chick            373
 omega            365
 pimp             345
 doll             343
 mistress         331
 wench            313
 misandrist       293
 john             277
 gigolo           258
 bitch            228
 tart             227
 volcel           206
 whore             74
 slut              40
 hooker            27
 harlot            11
 skank              6
 tramp              3
 streetwalker       2
 dtype: int64,
 'gender': gay            2116
 

In [12]:
stop_words = list(stopwords.words('english'))
stop_words.append(['rt', 'mkr', 'didn', 'bc', 'n', 'm', 
                  'im', 'll', 'y', 've', 'u', 'ur', 'don', 
                  'p', 't', 's', 'aren', 'kp', 'o', 'kat', 
                  'de', 're', 'amp', 'will', 'wa', 'e', 'like', 'yo', 'bc', 'amp'])

In [13]:
corpus = [(dword_freq[key],key) for key in list(dword_freq.keys()) if key not in stop_words]

In [14]:
corpus

[(man             2490
  guy             2380
  mansplain       1552
  bro             1131
  player          1116
  misogynist      1067
  manterrupt      1053
  incel           1041
  dude            1016
  escort           969
  manspread        918
  slayer           910
  slag             875
  feminazi         833
  ho               830
  cunt             726
  babe             707
  beta             695
  stud             559
  madam            545
  alpha            513
  bird             510
  beard            485
  fox              484
  chick            373
  omega            365
  pimp             345
  doll             343
  mistress         331
  wench            313
  misandrist       293
  john             277
  gigolo           258
  bitch            228
  tart             227
  volcel           206
  whore             74
  slut              40
  hooker            27
  harlot            11
  skank              6
  tramp              3
  streetwalker       2
  dtype: in

In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report

# collect labeled dataset of text samples and discrimination types
X = data['content']
y = data['disc_cat']

# preprocess text data by cleaning and vectorizing text
vectorizer = TfidfVectorizer(stop_words='english')
X_train = vectorizer.fit_transform(X)

# train classification model on preprocessed text data
clf = MultinomialNB()
clf.fit(X_train, y)




MultinomialNB()

In [20]:
# evaluate model performance on validation set
X_val = vectorizer.transform(X_val)
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

y_pred = clf.predict(X_val)
print('Accuracy:', accuracy_score(y_val, y_pred))
print('Precision:', precision_score(y_val, y_pred, average='weighted'))
print('Recall:', recall_score(y_val, y_pred, average='weighted'))
print('F1 score:', f1_score(y_val, y_pred, average='weighted'))
print('Confusion matrix:\n', confusion_matrix(y_val, y_pred))


TypeError: 'ellipsis' object is not iterable

In [15]:
dfs = []
for key in discriminatory_words:
    if key in data['disc_cat'].unique():
        # get word frequencies for current category
        dword_freq = get_top_discriminatory_words(data, discriminatory_words)
        # create DataFrame for current category
        df = pd.DataFrame({'word': list(dword_freq.keys()), 'freq': list(dword_freq.values())})
        df['category'] = key
        dfs.append(df)

# concatenate all DataFrames into a single DataFrame
tweets = pd.concat(dfs)

# sort the DataFrame by category and frequency
tweets = tweets.sort_values(by=['category', 'freq'], ascending=[True, False])

# reset the index of the DataFrame
tweets = tweets.reset_index(drop=True)

TypeError: unhashable type: 'Series'

# Reducing the Corpus

In [16]:
corpus.sort(reverse = True)

# keep the  most frequent words
corpus_freq = [(word[1],word[0]) for word in corpus[:50]] 
corpus_freq = corpus_freq[1:]
corpus_freq

ValueError: Can only compare identically-labeled Series objects

In [17]:

#create a DataFrame with two columns: 'word' and 'freq'
tweets = pd.DataFrame({'word': list(dword_freq.keys()), 'freq': list(dword_freq.values())})

# sort the DataFrame by frequency (descending)
tweets = tweets.sort_values(by='freq', ascending=False)

# reset the index of the DataFrame
tweets = tweets.reset_index(drop=True)

tweets

ValueError: Can only compare identically-labeled Series objects

In [ ]:
import nltk
from nltk.stem import WordNetLemmatizer

lem = WordNetLemmatizer()

def tweet_inspector(sentence, stop_words, words):

    # Decompose the review in words -> tokens
    tokens = nltk.word_tokenize(sentence)
    # Cleanup the tokens
    for i in range(len(tokens)):
        tokens[i] = tokens[i].lower()
        tokens[i] = re.sub(r'\W',' ',tokens[i]) # Replace everything non-alpahnumeric by ' '
        tokens[i] = re.sub(r'\s+','',tokens[i]) # Replace one or more whitespaces by  ' '
        tokens[i] = re.sub(r'\d+','',tokens[i]) # Replace one or more digits by  ' '
        tokens[i] = lem.lemmatize(tokens[i])
        
    # Droping tokens which are "stopwords" or empty
    tokens = [ token for token in tokens if (token not in stop_words and token != '')]

    # Initializing an empty dictionary of word frequencies for the corresponding review
    col_freq = {col:0 for col in words}
    
    # Filling the dictionary with word frequencies in the review
    for token in tokens:
        if token in words:
            col_freq[token] += 1

    return col_freq

In [ ]:
nltk.download('omw-1.4')
tweet_list = list( map(tweet_inspector, data['content'], 
                    [stop_words]*data.shape[0], [list(cols.keys())]*data.shape[0] ) )

tweet_list[:2]

In [ ]:
tweets = pd.DataFrame(tweet_list)

In [ ]:
tweets['disc_cat_num'] = data['disc_cat_num'].reset_index(drop=True)


In [ ]:
tweets

# X/Y split

In [ ]:
X= tweets.drop(columns=['disc_cat_num'])
y= tweets['disc_cat_num']

# train / test split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

X_train = pd.DataFrame(X_train, columns=X.columns)
X_test  = pd.DataFrame(X_test, columns=X.columns)

y_train = pd.DataFrame(y_train, columns =['disc_cat_num'])
y_test  = pd.DataFrame(y_test, columns =['disc_cat_num'])

In [ ]:
sns.countplot(x=y_train['disc_cat_num'])

In [ ]:
sns.countplot(x=y_test['disc_cat_num'])

# Class balancing

In [ ]:
from imblearn.under_sampling import RandomUnderSampler

rus = RandomUnderSampler(random_state=42)
X_train_res, y_train_res = rus.fit_resample(X_train, y_train)

X_train_res = pd.DataFrame(X_train_res, columns= X_train.columns)
y_train_res = pd.DataFrame(y_train_res, columns =['disc_cat_num'])

In [ ]:
sns.countplot(x=y_train_res['disc_cat_num'])

In [ ]:
y_train_res.value_counts()

# TF - IDF

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
tfidf=TfidfVectorizer(max_features=5000)

In [ ]:
x_train_tfidf=tfidf.fit_transform(X_train)
x_test_tfidf=tfidf.transform(X_test)

In [ ]:
print(x_train_tfidf)

In [ ]:
from sklearn.preprocessing import StandardScaler
scale=StandardScaler()
x_train_tfidf=x_train_tfidf.toarray()
scaled_x_train=scale.fit_transform(x_train_tfidf)
x_test_tfidf=x_test_tfidf.toarray()
scaled_x_test=scale.transform(x_test_tfidf)

# PCA

In [ ]:
# Reducing the number of features to check how the cumulative explained varience changes.
from sklearn.decomposition import PCA
pca=PCA()
reduced_train=pca.fit_transform(scaled_x_train)
pca.explained_variance_ratio_.size

In [ ]:
varience_explained=np.cumsum(pca.explained_variance_ratio_)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
plt.figure(figsize=(10,8))
plt.plot(varience_explained,color='b')
plt.xlabel("Number of components")
plt.ylabel("Cumulative explained variance")

In [ ]:
final_pca=PCA(0.9)
final_reduced_x_train=final_pca.fit_transform(x_train_tfidf)

In [ ]:
final_reduced_x_test=final_pca.transform(x_test_tfidf)

# Random Forest Classifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier



rf = RandomForestClassifier(max_depth=100, n_estimators=50)

y_train = y_train.astype('int')
y_test  = y_test.astype('int')

rf.fit(X_train, y_train)

y_train_pred_rf = rf.predict(X_train)
y_test_pred_rf  = rf.predict(X_test)

In [ ]:
def model_performance_class(y_train, y_pred_train, y_test, y_pred_test):

    from sklearn.metrics import cohen_kappa_score, classification_report 


    print("Results obtained for the TRAIN SET")
    print("==================================")
    print("The Cohen's Kappa is: {:.2f}".format(cohen_kappa_score(y_train, y_pred_train)))
    print(classification_report(y_train, y_pred_train))
    print("==================================")
    print("Results obtained for the TEST SET")
    print("The Cohen's Kappa is: {:.2f}".format(cohen_kappa_score(y_test, y_pred_test)))
    print(classification_report(y_test, y_pred_test))

In [ ]:
model_performance_class(y_train, y_train_pred_rf, y_test, y_test_pred_rf)

# Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression
log_model = LogisticRegression()
log_model.fit(X_train, y_train)

In [ ]:
print("Train Accuracy  : {:.2f} %".format(accuracy_score(log_model.predict(X_train), y_train)*100))
print("Test Accuracy   : {:.2f} %".format(accuracy_score(log_model.predict(X_test), y_test)*100))
print("Precision       : {:.2f} %".format(precision_score(log_model.predict(X_test), y_test,average='macro')*100))
print("Recall          : {:.2f} %".format(recall_score(log_model.predict(X_test), y_test,average='macro')*100))